In [ ]:
import base64
import hashlib
import hmac
import json
import os
import time
import requests
import urllib


# Refer to official documentation: https://www.xfyun.cn/doc/asr/ifasr_new/API.html#%E6%8E%A5%E5%8F%A3%E8%AF%B4%E6%98%8E
lfasr_host = 'https://raasr.xfyun.cn/v2/api'  # API host address (Required, obtained from console)

# API endpoints
api_upload = '/upload'         # File upload endpoint (Required)
api_get_result = '/getResult'  # Result query endpoint (Required)


# Please obtain the following information from the official platform at: https://console.xfyun.cn/services/lfasr
# Configuration from console (required)
my_appid = ""          # Application ID (Required, obtained from the console)
my_secret_key = ""   # Secret Key (Required, obtained from the console)

# Audio file configuration
my_upload_file_path = ""  # Audio path (Required, supports mp3,wav,pcm,aac,opus,flac,ogg,m4a,amr,speex（微信）,lyb,ac3,aac,ape,m4r,mp4,acc,wma)


class RequestApi(object):
    def __init__(self, appid, secret_key, upload_file_path):
        self.appid = appid
        self.secret_key = secret_key
        self.upload_file_path = upload_file_path
        self.ts = str(int(time.time()))
        self.signa = self.get_signa()

    def get_signa(self):
        appid = self.appid
        secret_key = self.secret_key
        m2 = hashlib.md5()
        m2.update((appid + self.ts).encode('utf-8'))
        md5 = m2.hexdigest()
        md5 = bytes(md5, encoding='utf-8')
        # 以secret_key为key, 上面的md5为msg， 使用hashlib.sha1加密结果为signa
        signa = hmac.new(secret_key.encode('utf-8'), md5, hashlib.sha1).digest()
        signa = base64.b64encode(signa)
        signa = str(signa, 'utf-8')
        return signa

    def upload(self):
        print("上传部分：")
        upload_file_path = self.upload_file_path
        file_len = os.path.getsize(upload_file_path)
        file_name = os.path.basename(upload_file_path)

        #  Refer to official documentation: https://www.xfyun.cn/doc/asr/ifasr_new/API.html#%E6%8E%A5%E5%8F%A3%E8%AF%B4%E6%98%8E
        param_dict = {}
        param_dict['appId'] = self.appid
        param_dict['signa'] = self.signa
        param_dict['ts'] = self.ts
        param_dict["fileSize"] = file_len
        param_dict["fileName"] = file_name
        param_dict["duration"] = "200"  # 使用音频的实际时长
        param_dict["language"] = "cn"  # 语种类型，例如 "cn" 为中文
        param_dict["callbackUrl"] = "http://yourcallbackurl.com"  # 设置回调URL
        param_dict["hotWord"] = "医学"  # 热词示例
        param_dict["audioMode"] = "fileStream"  # 文件流模式
        param_dict["standardWav"] = 0  # 是否标准pcm/wav
        param_dict["roleType"] = 1  # 开启角色分离
        param_dict["roleNum"] = 2  # 2 个人
        param_dict["pd"] = "medical"  # 领域个性化参数
        param_dict["transMode"] = 3  # 按整篇
        param_dict["eng_seg_max"] = 100  # 控制分段的最大字数（例如：100个字为一段）
        param_dict["eng_seg_min"] = 30   # 控制分段的最小字数（例如：30个字为一段）
        param_dict["eng_seg_weight"] = 0.02  # 控制分段字数的权重
        param_dict["eng_colloqproc"] = True  # 开启口语规整
        param_dict["eng_vad_mdn"] = 2  # 近场模式
        print("upload参数：", param_dict)
        data = open(upload_file_path, 'rb').read(file_len)

        response = requests.post(url=lfasr_host + api_upload + "?" + urllib.parse.urlencode(param_dict),
                                 headers={"Content-type": "application/json"}, data=data)
        print("upload_url:", response.request.url)
        result = json.loads(response.text)
        print("upload resp:", result)
        return result

    def get_result(self):
        uploadresp = self.upload()
        orderId = uploadresp['content']['orderId']
        param_dict = {}
        param_dict['appId'] = self.appid
        param_dict['signa'] = self.signa
        param_dict['ts'] = self.ts
        param_dict['orderId'] = orderId
        param_dict['resultType'] = "transfer,predict"
        print("")
        print("查询部分：")
        print("get result参数：", param_dict)
        status = 3
        # 建议使用回调的方式查询结果，查询接口有请求频率限制
        while status == 3:
            response = requests.post(url=lfasr_host + api_get_result + "?" + urllib.parse.urlencode(param_dict),
                                    headers={"Content-type": "application/json"})
            result = json.loads(response.text)
            print(result)  # Output the full response for debugging
            status = result['content']['orderInfo']['status']
            print("status=", status)
            if status == 4:  # Status 4 means result is ready
                break
            time.sleep(5)

        print("get_result resp:", result)

        # 解析识别结果并输出
        result_text = self.parse_result(result)
        
        # 保存为txt文件
        self.save_to_txt(result_text)

        return result

    def parse_result(self, result):
        # 解析转写结果
        dict_1 = json.loads(result['content']['orderResult'])
        text = ''
        current_role = None  # 用于记录当前的说话者角色
        for i in range(0, len(dict_1['lattice'])):
            json_1best = json.loads(dict_1['lattice'][i]['json_1best'])
            for item in json_1best['st']['rt']:
                role = json_1best['st'].get('rl', None)  # 提取角色
                # 如果当前角色与上一句不同，更新角色并标记
                if role != current_role:
                    current_role = role
                    text += f"[角色{role}] "  # 在新句子开始时标注角色
                
                for word in item['ws']:
                    word_text = word['cw'][0]['w']  # 获取识别的词语
                    text += word_text + ' '  # 添加识别的词语
                text += '\n'  # 每个句子换行

        return text

    def save_to_txt(self, text):
        # 保存为txt文件
        file_name = "transcription_result_with_roles.txt"
        with open(file_name, 'w', encoding='utf-8') as file:
            file.write(text)
        print(f"Transcription result saved to {file_name}")


# 输入讯飞开放平台的appid，secret_key和待转写的文件路径
if __name__ == '__main__':
    api = RequestApi(appid=my_appid,
                     secret_key=my_secret_key,
                     upload_file_path=my_upload_file_path)
    api.get_result()


上传部分：
upload参数： {'appId': '13f26487', 'signa': 'nPz5LIEJn/Oxrh5o8SN9o6kGp4k=', 'ts': '1739966973', 'fileSize': 42720092, 'fileName': '53d8490b303259a83d533646cfe3b639.mp4', 'duration': '200', 'language': 'cn', 'callbackUrl': 'http://yourcallbackurl.com', 'hotWord': '医学', 'audioMode': 'fileStream', 'standardWav': 0, 'roleType': 1, 'roleNum': 2, 'pd': 'medical', 'transMode': 3, 'eng_seg_max': 100, 'eng_seg_min': 30, 'eng_seg_weight': 0.02, 'eng_colloqproc': True, 'eng_vad_mdn': 2}
upload_url: https://raasr.xfyun.cn/v2/api/upload?appId=13f26487&signa=nPz5LIEJn%2FOxrh5o8SN9o6kGp4k%3D&ts=1739966973&fileSize=42720092&fileName=53d8490b303259a83d533646cfe3b639.mp4&duration=200&language=cn&callbackUrl=http%3A%2F%2Fyourcallbackurl.com&hotWord=%E5%8C%BB%E5%AD%A6&audioMode=fileStream&standardWav=0&roleType=1&roleNum=2&pd=medical&transMode=3&eng_seg_max=100&eng_seg_min=30&eng_seg_weight=0.02&eng_colloqproc=True&eng_vad_mdn=2
upload resp: {'code': '000000', 'descInfo': 'success', 'content': {'orderI